In [11]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [3]:
dataset = 'Retina'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['Annotation'].to_numpy()

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)
#adata.write_h5ad(f'data/{dataset}_dpp.h5ad')

In [4]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Mcm6','Cdk1','Esco2','Cenpa']

# Discrete VeloVAE

In [6]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.DVAE(adata, 
               tmax=20, 
               dim_z=5, 
               device='cuda:0',
               init_method='random')

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 1, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.26, Min=0.00, Mean=0.97
Library scale (S): Max=6.23, Min=0.27, Mean=1.10
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 15, test iteration: 28
*********       Stage 1: Early Stop Triggered at epoch 378.       *********
*********                      Stage  2                       *********


  0%|          | 0/2587 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.015
Average Set Size: 82
*********       Stage 2: Early Stop Triggered at epoch 1515.       *********
*********              Finished. Total Time =   0 h :  5 m : 46 s             *********
Final: Train ELBO = -1883.618,           Test ELBO = -1964.664
       Training MSE = 1.804, Test MSE = 2.031


# Full VB

In [7]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.DVAEFullVB(adata, 
                        tmax=20, 
                        dim_z=5, 
                        device='cuda:0',
                        init_method='steady')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 1, 0
Initialization using the steady-state and dynamical models.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=4.26, Min=0.00, Mean=0.97
Library scale (S): Max=6.23, Min=0.27, Mean=1.10
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 15, test iteration: 28
*********       Stage 1: Early Stop Triggered at epoch 290.       *********
*********                      Stage  2                       *********


  0%|          | 0/2587 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.006
Average Set Size: 91
*********       Stage 2: Early Stop Triggered at epoch 529.       *********
*********              Finished. Total Time =   0 h :  2 m : 30 s             *********
Final: Train ELBO = -1994.358,           Test ELBO = -2086.944
       Training MSE = 2.208, Test MSE = 2.446


# Evaluation

In [ ]:
cluster_edges = [('Neuroblast', 'PR'), ('Neuroblast', 'AC/HC'), ('Neuroblast', 'RGC')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)